In [ ]:
#Model 4 with required results
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle

# Load and preprocess the Fashion MNIST data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = tf.image.resize(x_train, [96, 96])
x_test = tf.image.resize(x_test, [96, 96])
x_train = tf.image.grayscale_to_rgb(tf.convert_to_tensor(x_train))
x_test = tf.image.grayscale_to_rgb(tf.convert_to_tensor(x_test))

# Load the pre-trained MobileNetV2 model
base_model = MobileNetV2(input_shape=(96, 96, 3), include_top=False, weights='imagenet', pooling='avg')

# Make the base model non-trainable
base_model.trainable = False

# Adding RNN and output layers
intermediate_output = base_model.output
sequence_output = layers.RepeatVector(3)(intermediate_output)
sequence_output = layers.SimpleRNN(128)(sequence_output)
output = layers.Dense(10, activation='softmax')(sequence_output)

# Create the complete model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert labels to categorical format
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

# Train the model
history = model.fit(x_train, y_train_cat, epochs=10, validation_data=(x_test, y_test_cat))

# Evaluate the model
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_classes)
print('Confusion Matrix:\n', cm)

# Classification Report
cr = classification_report(y_test, y_pred_classes)
print('Classification Report:\n', cr)

# ROC Curve
y_test_bin = label_binarize(y_test, classes=[i for i in range(10)])
n_classes = y_test_bin.shape[1]
y_pred_bin = label_binarize(y_pred_classes, classes=[i for i in range(10)])

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_pred_bin[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot all ROC curves
colors = cycle(['blue', 'red', 'green', 'cyan', 'magenta', 'yellow', 'black', 'pink', 'lightgreen', 'lightblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()
